<a href="https://colab.research.google.com/github/Drisya-Ponmari/UST_INTERN/blob/master/%20Question_answer_model%20/QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Jul 21 06:07:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/huggingface/transformers.git
!pip install -U ./transformers
!pip install git+https://github.com/huggingface/nlp.git

fatal: destination path 'transformers' already exists and is not an empty directory.
Processing ./transformers
  Created wheel for transformers: filename=transformers-3.0.2-cp36-none-any.whl size=793805 sha256=36bd9e6b9b351cea6c62fa39995684efdabae5471b3f51066a9fe138973bde06
  Stored in directory: /tmp/pip-ephem-wheel-cache-vt3yk8kn/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2
  Cloning https://github.com/huggingface/nlp.git to /tmp/pip-req-build-prg97ni4
  Running command git clone -q https://github.com/huggingface/nlp.git /tmp/pip-req-build-prg97ni4
  Created wheel for nlp: filename=nlp-0.3.0-cp36-none-any.whl size=114550 sha256=039f8c88fbae0be4faf1cc8bea1d0578f5d39fa39dc207d0a1270c57c975faaa
  Stored in directory: /tmp/pip-ephem-wheel-cache-1au5sbuc/wheels/a2/98/bd/15f115cb85f1a049a96b915aa14466d

##IMPORT LIBRARIES##

In [ ]:
import torch
import nlp
from transformers import LongformerTokenizerFast

##DOWNLOADING SQUAD V2 DATASET##

In [ ]:
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')

##EXPLORING DATASET##

In [ ]:
#train_dataset = json.load(open('train-v2.0.json'))
#test_dataset  = json.load(open('dev-v2.0.json'))

**The dataset contains two section context and question answer.**
Overview of train and test json file:

**version**

          ->->....


**data**

        ->->title

                  ->->->...

        ->->paragraphs

                      ->->->quas

                                ->->->->question

                                ->->->->answer

                                            ->->->->->text

                                            ->->->->->answer_start

                                            ->->->->->is_impossible

                  ->->->context

                                ->->->->...

In [ ]:
a = "When did Beyonce start becoming popular?"
b = "Beyonc\u00e9 Giselle Knowles-Carter (/bi\u02d0\u02c8j\u0252nse\u026a/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all the time. Their hiatus saw the release of Beyonc\u00e9's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles \"Crazy in Love\" and \"Baby Boy"
c = 269
d = "in the late 1990s"
e = ' '

##SET UP **BERT** TOKENIZER##

In [ ]:
from transformers import LongformerTokenizerFast
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')

##TOKENISING##


In [ ]:
def get_correct_alignement(context, answer):
    """ Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here. """
    gold_text = answer['text'][0]
    start_idx = answer['answer_start'][0]
    end_idx = start_idx + len(gold_text)
    if context[start_idx:end_idx] == gold_text:
        return start_idx, end_idx       # When the gold label position is good
    elif context[start_idx-1:end_idx-1] == gold_text:
        return start_idx-1, end_idx-1   # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
        return start_idx-2, end_idx-2   # When the gold label is off by two character
    else:
        raise ValueError()

# Tokenize our training dataset
def convert_to_features(example):
    # Tokenize contexts and questions (as pairs of inputs)
    input_pairs = [example['question'], example['context']]
    
    encodings = tokenizer.encode_plus(str(example['question']),str(example['context']), pad_to_max_length=True, max_length=512)
    context_encodings = tokenizer.encode_plus(example['context'])
    

    # Compute start and end tokens for labels using Transformers's fast tokenizers alignement methodes.
    # this will give us the position of answer span in the context text
    start_idx, end_idx = get_correct_alignement(example['context'], example['answers'])
    start_positions_context = context_encodings.char_to_token(start_idx)
    end_positions_context = context_encodings.char_to_token(end_idx-1)

    # here we will compute the start and end position of the answer in the whole example
    # as the example is encoded like this <s> question</s></s> context</s>
    # and we know the postion of the answer in the context
    # we can just find out the index of the sep token and then add that to position + 1 (+1 because there are two sep tokens)
    # this will give us the position of the answer span in whole example 
    sep_idx = encodings['input_ids'].index(tokenizer.sep_token_id)
    start_positions = start_positions_context + sep_idx + 1
    end_positions = end_positions_context + sep_idx + 1

    if end_positions > 512:
      start_positions, end_positions = 0, 0

    encodings.update({'start_positions': start_positions,
                      'end_positions': end_positions,
                      'attention_mask': encodings['attention_mask']})
    return encodings

In [ ]:

train_dataset  = nlp.load_dataset('squad', split='train[:2%]')
valid_dataset = nlp.load_dataset('squad', split='validation[:1%]')
 

In [ ]:
print(len(train_dataset))

1752


In [ ]:
print(len(valid_dataset))

106


In [ ]:
'''x = train_dataset[0:100]
j =10;
for i in train_dataset:
  if(j<0):
    break
  print(i)
  j = j-1
print(type(train_dataset))'''

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}
{'id': '5733be284776f4190066117f', 'title': 'Universit

In [ ]:
train_dataset = train_dataset.map(convert_to_features)
valid_dataset = valid_dataset.map(convert_to_features, load_from_cache_file=False)


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

In [ ]:
# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'attention_mask', 'start_positions', 'end_positions']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [ ]:
len(train_dataset), len(valid_dataset)

(1752, 106)

In [ ]:
torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')

In [ ]:
# Setting up the device for GPU usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from transformers import LongformerForQuestionAnswering, LongformerTokenizerFast, EvalPrediction
model = LongformerForQuestionAnswering.from_pretrained(
    "allenai/longformer-base-4096",
)
model.to(device)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForQuestionAnswering were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mo

LongformerForQuestionAnswering(
  (longformer): LongformerModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_globa

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total # of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset            # evaluation dataset
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [24]:
%load_ext tensorboard

In [25]:
trainer.save_model("./Question-Answering")
tokenizer.save_pretrained("./Question-Answering")

('./Question-Answering/vocab.json',
 './Question-Answering/merges.txt',
 './Question-Answering/special_tokens_map.json',
 './Question-Answering/added_tokens.json')

In [27]:
!transformers-cli login

2020-07-21 07:46:51.273763: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: ponmari
Password: 
Login successful
Your token: HdqHRfdRlVBnqJRWUuJTjgDGFUruPlNvEcCfwqImOhmyzcwYEvdjkSkeIcqWssYFHlekQMlDjAyOrghwLEwhNqcmqrTOTbyQhtFSIZIpDnYVQwukuaXJbfBkkoJhVYdl 

Your token has been saved to /root/.huggingface/token


In [29]:
!transformers-cli upload Question-Answering/

2020-07-21 07:55:36.617052: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
About to upload file /content/Question-Answering/tokenizer_config.json to S3 under filename Question-Answering/tokenizer_config.json and namespace ponmari
About to upload file /content/Question-Answering/training_args.bin to S3 under filename Question-Answering/training_args.bin and namespace ponmari
About to upload file /content/Question-Answering/pytorch_model.bin to S3 under filename Question-Answering/pytorch_model.bin and namespace ponmari
About to upload file /content/Question-Answering/config.json to S3 under filename Question-Answering/config.json and namespace ponmari
About to upload file /content/Question-Answering/vocab.json to S3 under filename Question-Answering/vocab.json and namespace ponmari
About to upload file /content/Question-Answering/special_tokens_map.json to S3 under filename Question-Answering/special_tokens_map.json 